# SIFT Feature Detection and Matching Demo

This notebook demonstrates how to use the custom SIFT implementation for feature detection, description, and matching between images.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2  # Only for basic image operations
# from sift_useful import *

## 0.Useful Functions

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import tqdm

def my_conv2d(kernel, img, conv_type="full"):
    """
    Implement 2D image convolution
    :param kernel: float/int array, shape: (x, x)
    :param img: float/int array, shape: (height, width)
    :param conv_type: str, convolution padding choices, should in ['full', 'same', 'valid']
    :return conv results, numpy array
    """
    k_h, k_w = kernel.shape
    i_h, i_w = img.shape

    kernel = np.rot90(kernel, 2)

    # Calculate output dimensions based on conv_type
    half_k_h = k_h // 2
    half_k_w = k_w // 2
    if conv_type == "full":
        pad_h = k_h - 1
        pad_w = k_w - 1
        out_h = i_h + pad_h
        out_w = i_w + pad_w
    elif conv_type == "same":
        pad_h = k_h // 2
        pad_w = k_w // 2
        out_h = i_h
        out_w = i_w
    elif conv_type == "valid":
        pad_h = pad_w = 0
        out_h = i_h - k_h + 1
        out_w = i_w - k_w + 1
    else:
        raise ValueError("conv_type must be 'full', 'same', or 'valid'")

    # Pad the image, put it in the center with 0 outside
    padded_img = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)

    # Perform convolution
    result = np.zeros((out_h, out_w))
    for i in range(out_h):
        for j in range(out_w):
            img_region = padded_img[i : i + k_h, j : j + k_w]
            result[i, j] = np.sum(img_region * kernel)
            
    pass
    # =========================================================================================================
    return result

def my_conv3d(kernel, img, conv_type="full"):
    """
    Implement 3D image convolution
    :param kernel: float/int array, shape: (ci, h, w, co)
    :param img: float/int array, shape: (height, width, channel)
    :param conv_type: str, convolution padding choices, should in ['full', 'same', 'valid']
    :return conv results, numpy array
    """
    assert len(kernel.shape) == 4 and len(img.shape) == 3, "The dimensions of kernel and img should be 3."
    ci, k_h, k_w, co = kernel.shape
    i_h, i_w, i_c = img.shape

    result = np.zeros((i_h, i_w, i_c))
    for c_i in range(ci):
      for c_o in range(co):
        result[:, :, c_o] = my_conv2d(kernel[c_i, :, :, c_o], img[:, :, c_o], conv_type=conv_type)
        
    return result

def generate_gaussian_kernel(size, sigma = 1):
    '''This function generates a 2D Gaussian kernel. And it output a 2D numpy array whose values are the Gaussian values between 0 and 1.'''
    # generate an axis
    x = np.linspace(-size // 2, size // 2, size)
    gauss_x = np.exp(-x ** 2 / (2 * sigma ** 2))

    # generate a 2D gaussian kernel and normalize
    gauss_2d = np.outer(gauss_x, gauss_x)
    gauss_2d /= gauss_2d.sum()
    return gauss_2d

def gaussian_filter(img, kernel_size, sigma=1):
		"""
		Implement Gaussian filter
		:param img: float/int array, shape: (height, width)
		:param kernel_size: int, the size of the kernel, should be odd number
		:param sigma: float, standard deviation of the Gaussian distribution
		:return filtered image, numpy array
		"""
		assert kernel_size % 2 == 1, "kernel size should be odd number"
		
		# Generate Gaussian kernel
		kernel = generate_gaussian_kernel(kernel_size, sigma=sigma)

		# Perform convolution using my_conv2d function
		filtered_img = my_conv2d(kernel, img, conv_type="same")
		
		return filtered_img

In [ ]:
def create_gaussian_pyramid(image, num_octaves, scales_per_octave, sigma0=1.6):
    """
    Create a Gaussian pyramid with multiple octaves and scales.
    
    Args:
        image: Input grayscale image
        num_octaves: Number of octaves
        scales_per_octave: Number of scales per octave
        sigma0: Initial sigma
        
    Returns:
        gaussian_pyramid: List of octaves, each containing Gaussian blurred images
    """
    # # change to [0, 1] range
    # if image.max() > 1:
    #     image = image / 255.0

    k = 2 ** (1.0 / scales_per_octave)
    gaussian_pyramid = []
    
    # For each octave
    for octave in range(num_octaves):
        octave_images = []
        
        # Downsample image for this octave
        if octave == 0:
            octave_base = image.copy()
        else:
            octave_base = cv2.resize(gaussian_pyramid[-1][0], 
                                     (gaussian_pyramid[-1][0].shape[1] // 2, 
                                     gaussian_pyramid[-1][0].shape[0] // 2))
        
        # Create scales for this octave
        for scale in range(scales_per_octave + 3):
            sigma = sigma0 * (k ** scale)
            kernel_size = int(2 * np.ceil(2 * sigma) + 1)
            blurred = gaussian_filter(octave_base, kernel_size, sigma=sigma)
            octave_images.append(blurred)
            
        gaussian_pyramid.append(octave_images)
        
    
    # show the gaussian pyramid, one plot for each octave, total plot number is num_octaves
    for i in range(num_octaves):
        plt.figure(figsize=(100, 15))
        for j in range(scales_per_octave + 3):
            plt.subplot(1, scales_per_octave + 3, j + 1)
            plt.imshow(gaussian_pyramid[i][j], cmap='gray')
            # plt.axis('off')
        print("Octave: ", i, " Scale: ", scales_per_octave + 3, " Shape: ", gaussian_pyramid[i][j].shape)
        sigma_box = [sigma0 * (k ** scale) for scale in range(scales_per_octave + 2)]
        print("Sigma: ", sigma_box)
        #print("Gaussian kernel: size", kernel_size, "\n", generate_gaussian_kernel(kernel_size, sigma=sigma_box[0] * 5))
        plt.tight_layout()
        plt.show()


    return gaussian_pyramid

def create_dog_pyramid(gaussian_pyramid):
    """
    Create Difference-of-Gaussian (DoG) pyramid from Gaussian pyramid.
    
    Args:
        gaussian_pyramid: Gaussian pyramid (list of octaves)
        
    Returns:
        dog_pyramid: List of DoG octaves containing difference images
    """
    dog_pyramid = []
    initial_shape = gaussian_pyramid[0][0].shape
    print("Initial Shape: ", initial_shape)
    print("Initial Image: ", gaussian_pyramid[0][0])
    print("Second Image: ", gaussian_pyramid[0][1])
    
    for octave_images in gaussian_pyramid:
        # Verify valid input structure
        if len(octave_images) < 2:
            raise ValueError("Octave must contain at least 2 Gaussian images")
            
        dog_images = []
        # Generate (n-1) DoG images per octave and remember to resize the image to the same size
        for i in range(1, len(octave_images)):
            dog = octave_images[i] - octave_images[i-1]
            if i == 1:
              print("max: ", np.max(dog), "shape: ", dog.shape, "DoG: ", dog)
            #dog = cv2.resize(dog, (initial_shape[1], initial_shape[0]))
            # change to [0, 1] range
            dog = (dog - np.min(dog)) / (np.max(dog) - np.min(dog)) 
            if i == 1:
              print("max: ", np.max(dog), "shape: ", dog.shape, "DoG: ", dog)
            dog_images.append(dog)
        dog_pyramid.append(dog_images)
    
    # show the DoG pyramid, one plot for each octave, total plot number is num_octaves
    #'''
    for i in range(len(dog_pyramid)):
        plt.figure(figsize=(100, 15))
        for j in range(len(dog_pyramid[i])):
            plt.subplot(1, len(dog_pyramid[i]), j + 1)
            plt.imshow(dog_pyramid[i][j], cmap='gray')
						# plt.axis('off')
        print("Octave: ", i, " Scale: ", len(dog_pyramid[i]), " Shape: ", dog_pyramid[i][j].shape)
        plt.tight_layout()
        plt.show()
    #'''

    return dog_pyramid


def detect_keypoints2(dog_pyramid, contrast_threshold=0.5, edge_threshold=10):
    """
    Detect keypoints in the DoG pyramid with sub-pixel refinement.
    
    Args:
        dog_pyramid: DoG pyramid from create_dog_pyramid()
        contrast_threshold: Minimum contrast (normalized to [0,1])
        edge_threshold: Eigenvalue ratio threshold (usually 10)
        
    Returns:
        keypoints: List of keypoints as (octave, scale, y, x)
    """
    keypoints = []
    #contrast_threshold = contrast_threshold * 255  # Scale to pixel values
    
    total_keypoints_num = 0
    for octave_idx, dog_octave in enumerate(dog_pyramid):
        # Require at least 3 DoG images for scale-space extremum
        if len(dog_octave) < 3:
            continue
        
        octave_num = 0
        for scale_idx in range(1, len(dog_octave)-1):
            prev_dog = dog_octave[scale_idx-1]
            curr_dog = dog_octave[scale_idx]
            next_dog = dog_octave[scale_idx+1]
            
            # Iterate through interior pixels (excluding 1-pixel border)
            height, width = curr_dog.shape
            for i in range(1, height-1):
                for j in range(1, width-1):
                    # Current pixel value
                    val = curr_dog[i, j]

                    
                    # 3D neighborhood check (3x3x3 cube)
                    neighborhood = np.concatenate([
                        prev_dog[i-1:i+2, j-1:j+2].flatten(),
                        curr_dog[i-1:i+2, j-1:j+2].flatten(),
                        next_dog[i-1:i+2, j-1:j+2].flatten()
                    ])  
                    neighborhood = np.delete(neighborhood, 13)  # Remove the center element
                    is_max = val > np.max(neighborhood)
                    is_min = val < np.min(neighborhood)

                    if not (is_max or is_min):
                        continue

                    # Edge response check using Hessian matrix
                    # Second derivatives (central differences)
                    dx = (curr_dog[i, j+1] - curr_dog[i, j-1]) / 2.0
                    dy = (curr_dog[i+1, j] - curr_dog[i-1, j]) / 2.0
                    dxx = curr_dog[i, j+1] + curr_dog[i, j-1] - 2*val
                    dyy = curr_dog[i+1, j] + curr_dog[i-1, j] - 2*val
                    dxy = (curr_dog[i+1, j+1] - curr_dog[i+1, j-1] - 
                          curr_dog[i-1, j+1] + curr_dog[i-1, j-1]) / 4.0
                    
                     # Solve for offset
                    gradient = np.array([dx, dy])
                    hessian = np.array([[dxx, dxy], [dxy, dyy]])
                    
                    try:
                        offset = -np.linalg.lstsq(hessian, gradient, rcond=None)[0]
                    except np.linalg.LinAlgError:
                        continue
                        
                    if np.abs(offset[0]) > 0.5 or np.abs(offset[1]) > 0.5:
                        continue  # Reject unstable refinements

                    # 3. Contrast thresholding AFTER refinement
                    contrast = curr_dog[i, j] + 0.5 * np.dot(gradient, offset)
                    if np.abs(contrast) < contrast_threshold:
                        continue

                    # 4. Edge rejection
                    tr = dxx + dyy
                    det = dxx * dyy - dxy**2
                    if det <= 0 or tr**2 * edge_threshold >= (edge_threshold + 1)**2 * det:
                        continue

                    # Store refined coordinates
                    keypoints.append((
                        octave_idx,
                        scale_idx,
                        i + offset[1],  # y-coordinate
                        j + offset[0]   # x-coordinate
                    ))
                    octave_num = octave_num + 1
                    
        print("find keypoints in octave: ", octave_idx, " keypoints number: ", octave_num)
        total_keypoints_num = total_keypoints_num + octave_num
    print("total keypoints number: ", total_keypoints_num)
                
    if len(keypoints) < 100:
        print("Warning: Very few keypoints detected, less than 100")
        print("redo the keypoints detection...", " last contrast_threshold: ", contrast_threshold)
        keypoints = detect_keypoints2(dog_pyramid, contrast_threshold=0.8* contrast_threshold, 
                                      edge_threshold=10)
        
    if len(keypoints) > 4000 and contrast_threshold < 0.9:
        print("Warning: Too many keypoints detected, more than 4000")
        print("redo the keypoints detection...", "last contrast_threshold: ", contrast_threshold)
        keypoints = detect_keypoints2(dog_pyramid, contrast_threshold=1.05 * contrast_threshold, edge_threshold=10)
        
    last_detected_number = len(keypoints)
    return keypoints


def compute_orientations2(gaussian_pyramid, keypoints, num_bins=36, sigma0=1.4):
    """
    Compute dominant orientations for each keypoint.
    
    Args:
        gaussian_pyramid: Gaussian pyramid
        keypoints: List of keypoints (octave, scale, y, x)
        num_bins: Number of orientation bins
        
    Returns:
        oriented_keypoints: List of keypoints with orientation (octave, scale, y, x, orientation)
    """
    oriented_keypoints = []
    
    for keypoint in keypoints:
        octave_idx, scale_idx, refine_y, refine_x = keypoint
        img = gaussian_pyramid[octave_idx][scale_idx]
        scales_per_octave = len(gaussian_pyramid[0]) - 2
        x = int(refine_x)
        y = int(refine_y)
        
        # Create histogram of orientations
        histogram = np.zeros(num_bins)
        # Gaussian weighting sigma0=1.6
        k = 2 ** (1.0 / scales_per_octave)
        sigma = sigma0 * (k ** scale_idx)
        radius = int(3 * sigma)
        
        for i in range(-radius, radius + 1):
            yi = y + i
            if yi <= 0 or yi >= img.shape[0] - 1:
                continue
                
            for j in range(-radius, radius + 1):
                xi = x + j
                if xi <= 0 or xi >= img.shape[1] - 1:
                    continue
                
                # Compute gradient
                dx = img[yi, xi+1] - img[yi, xi-1]
                dy = img[yi+1, xi] - img[yi-1, xi]
                
                # Compute magnitude and orientation
                magnitude = np.sqrt(dx**2 + dy**2)
                orientation = np.arctan2(dy, dx) % (2 * np.pi)
                
                # Weight by magnitude and distance from center
                weight = magnitude * np.exp(-(i**2 + j**2) / (2 * sigma**2))
                
                # Add to histogram
                bin_idx = int(orientation / (2 * np.pi) * num_bins) % num_bins
                histogram[bin_idx] += weight
        
        # Smooth histogram
        histogram = np.roll(histogram, 1) + histogram + np.roll(histogram, -1)
        histogram = histogram / 3.0
        
        # Find peaks in histogram
        threshold = 0.8 * np.max(histogram)
        for bin_idx in range(num_bins):
            if (histogram[bin_idx] > histogram[(bin_idx-1) % num_bins] and
                histogram[bin_idx] > histogram[(bin_idx+1) % num_bins] and
                histogram[bin_idx] >= threshold):
                
                # Convert bin index to angle (in radians)
                angle = bin_idx * 2 * np.pi / num_bins
                oriented_keypoints.append((octave_idx, scale_idx, refine_y, refine_x, angle))
    
    return oriented_keypoints


def compute_descriptors2(gaussian_pyramid, oriented_keypoints, descriptor_size=4, num_bins=8, sigma0=1.4):
    """
    Compute SIFT descriptors for keypoints.
    
    Args:
        gaussian_pyramid: Gaussian pyramid
        oriented_keypoints: List of keypoints with orientation
        descriptor_size: Size of descriptor grid (e.g., 4 means 4x4 grid)
        num_bins: Number of orientation bins per grid cell
        
    Returns:
        keypoints: List of keypoint locations (x, y, scale, orientation)
        descriptors: List of descriptors
    """
    # This function do these things:
    # 1. Compute the window size * size , size is 4 * 3 *sigma as int
    # 2. Get the orientation of the keypoint from the oriented_keypoints
    # 3. slice the window into 4 * 4 cells, and each cell has 8 bins, so the descriptor is 4 * 4 * 8, each cell size is 3 * sigma as int
    # 4. For each cell, calculate the gradient magnitude and orientation, and then weight by magnitude and distance from center, according to the orientation, add to the corresponding bin, here we should add the orientation to histogram bins
		# 5. Threshold and normalize for illumination invariance
    # 6. Add the bins to the descriptor
    # 7. Add the descriptor to the descriptors_list, so is the keypoints_list
    keypoints_list = []
    descriptors_list = []
    
    for keypoint in oriented_keypoints:
        octave_idx, scale_idx, refine_y, refine_x, angle = keypoint
        img = gaussian_pyramid[octave_idx][scale_idx]
        scales_per_octave = len(gaussian_pyramid[0]) - 2
        x = int(refine_x)
        y = int(refine_y)
        
        scale = 2 ** octave_idx
        k = 2 ** (1.0 / scales_per_octave)
        sigma = sigma0 * (k  ** scale_idx)
        
        cos_angle = np.cos(angle)
        sin_angle = np.sin(angle)
        
        cell_size = int(3 * sigma)
        half_width = (descriptor_size * cell_size) // 2
        
        descriptor = np.zeros((descriptor_size, descriptor_size, num_bins))
        
        for cell_i in range(descriptor_size):
            for cell_j in range(descriptor_size):
                for i in range(cell_size):
                    for j in range(cell_size):
                        # compute the rotated coordinates
                        u = (cell_i - descriptor_size/2) * cell_size + i
                        v = (cell_j - descriptor_size/2) * cell_size + j
                        x_rot = u * cos_angle - v * sin_angle
                        y_rot = u * sin_angle + v * cos_angle
                        sample_x = x + x_rot
                        sample_y = y + y_rot
                        
                        if sample_x < 1 or sample_x >= img.shape[1]-1 or sample_y < 1 or sample_y >= img.shape[0]-1:
                            continue
                        
                        dx = img[int(sample_y), int(sample_x)+1] - img[int(sample_y), int(sample_x)-1]
                        dy = img[int(sample_y)+1, int(sample_x)] - img[int(sample_y)-1, int(sample_x)]
                        magnitude = np.sqrt(dx**2 + dy**2)
                        orientation = (np.arctan2(dy, dx) - angle) % (2 * np.pi)
                        
                        bin_idx = int(orientation / (2 * np.pi) * num_bins) % num_bins
                        weight = magnitude * np.exp(-((i - cell_size//2)**2 + (j - cell_size//2)**2) / (2 * (0.5 * cell_size)**2))
                        descriptor[cell_i, cell_j, bin_idx] += weight
        
        flat_descriptor = descriptor.flatten()
        threshold = 0.2 * np.linalg.norm(flat_descriptor)
        flat_descriptor = np.minimum(flat_descriptor, threshold)
        norm = np.linalg.norm(flat_descriptor)
        if norm > 0:
            flat_descriptor /= norm
        
        keypoints_list.append((int(refine_x * scale), int(refine_y * scale), scale, angle))
        descriptors_list.append(flat_descriptor)
    
    return keypoints_list, np.array(descriptors_list)
   							         
                
def match_descriptors(desc1, desc2, ratio_threshold=0.75):
    """
    Match descriptors using ratio test.
    
    Args:
        desc1: First set of descriptors
        desc2: Second set of descriptors
        ratio_threshold: Ratio test threshold
        
    Returns:
        matches: List of matches (idx1, idx2)
    """
    matches = []
    
    for i, descriptor in enumerate(desc1):
        # Compute distances to all descriptors in desc2
        distances = []
        for descriptor2 in desc2:
            diff = descriptor - descriptor2
            distance = np.sqrt(np.sum(diff**2))
            distances.append(distance)
        
        # Find indices of two closest matches
        idx = np.argsort(distances)
        
        # Apply ratio test
        if distances[idx[0]] < ratio_threshold * distances[idx[1]]:
            matches.append((i, idx[0]))
    
    return matches

from homography_manual import find_homography_manual, compute_transformed_coordinates, warp_perspective_manual
def ransac_match(keypoints1, keypoints2, matches, num_iterations=1000, inlier_threshold=7):
    """
    Match keypoints between two images using RANSAC.
    
    Args:
        keypoints1: List of keypoints from first image
        descriptors1: List of descriptors from first image
        keypoints2: List of keypoints from second image
        descriptors2: List of descriptors from second image
        matches: List of matches (idx1, idx2)
        num_iterations: Number of RANSAC iterations        
        inlier_threshold: Threshold for inliers    

    Returns:    
        best_inliers: List of best inlier matches
        best_homography: Best homography matrix
    """
    import random
    
    # Need at least 4 points to compute homography
    if len(matches) < 4:
        return [], None
    
    # Extract matched points
    src_pts = np.float32([keypoints1[match[0]][:2] for match in matches])
    dst_pts = np.float32([keypoints2[match[1]][:2] for match in matches])
    
    best_inliers = []
    best_homography = None
    
    for _ in range(num_iterations):
        # Randomly select 4 matches
        sample_indices = random.sample(range(len(matches)), 4)
        
        # Extract the points from these matches
        src_sample = np.float32([src_pts[i] for i in sample_indices])
        dst_sample = np.float32([dst_pts[i] for i in sample_indices])
        
        try:
            # Compute homography using the 4 points
            homography = find_homography_manual(src_sample, dst_sample)
            
            # Skip if homography couldn't be computed
            if homography is None:
                continue
                
            # Count inliers
            inliers = []
            
            for i, (src, dst) in enumerate(zip(src_pts, dst_pts)):
                # Apply homography to source point
                src_transformed = np.dot(homography, np.array([src[0], src[1], 1]))
                if src_transformed[2] == 0:  # Check for division by zero
                    continue
                    
                # Convert to (x, y) coordinates
                src_transformed = src_transformed[:2] / src_transformed[2]
                
                # Calculate distance
                dist = np.sqrt(np.sum((src_transformed - dst)**2))
                
                # Check if it's an inlier
                if dist < inlier_threshold:
                    inliers.append(matches[i])
            
            # Update best result if this has more inliers
            if len(inliers) > len(best_inliers):
                best_inliers = inliers
                best_homography = homography
                
        except Exception as e:
            # Skip this iteration if there's an error
            continue
    # if we have enough inliers, we can compute the homography again
    if len(best_inliers) >= 4:
        src_pts = np.float32([keypoints1[match[0]][:2] for match in best_inliers])
        dst_pts = np.float32([keypoints2[match[1]][:2] for match in best_inliers])
        best_homography = find_homography_manual(src_pts, dst_pts)
    return best_inliers, best_homography
              
        
def visualize_keypoints(image, keypoints):
	"""
	Visualize keypoints on an image.
	
	Args:
		image: Input image
		keypoints: List of keypoints (x, y, scale, orientation)
	"""
	plt.figure(figsize=(10, 8))
	plt.imshow(image, cmap='gray')
	
	# Create a color cycle for different keypoints
	colors = plt.cm.hsv(np.linspace(0, 1, len(keypoints)))
	
	for i, kp in enumerate(keypoints):
		x, y, scale, orientation = kp
		
		radius = scale * 3
		
		# Draw keypoint circle with unique color
		circle = plt.Circle((x, y), radius, fill=False, color=colors[i])
		plt.gca().add_patch(circle)
		
		# Draw orientation line with same color
		line_x = x + radius * np.cos(orientation)
		line_y = y + radius * np.sin(orientation)
		plt.plot([x, line_x], [y, line_y], color=colors[i])
	
	plt.axis('off')
	plt.title("keypoints num: " + str(len(keypoints))) # Use f-string formatting
	plt.tight_layout()
	plt.show()

def visualize_matches(img1, kp1, img2, kp2, matches):
	"""
	Visualize matches between two images.
	
	Args:
		img1: First image
		kp1: Keypoints in first image
		img2: Second image
		kp2: Keypoints in second image
		matches: List of matches (idx1, idx2)
	"""
	# Create a new image with both images side by side
	h1, w1 = img1.shape[:2]
	h2, w2 = img2.shape[:2]
	
	h = max(h1, h2)
	w = w1 + w2
	
	vis = np.zeros((h, w), dtype=np.uint8)
	vis[:h1, :w1] = img1 if len(img1.shape) == 2 else cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
	vis[:h2, w1:w1+w2] = img2 if len(img2.shape) == 2 else cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
	
	plt.figure(figsize=(12, 8))
	plt.imshow(vis, cmap='gray')
	
	# Create color map
	colors = plt.cm.hsv(np.linspace(0, 1, len(matches)))
	
	# Draw lines between matches
	for i, match in enumerate(matches):
		idx1, idx2 = match
		x1, y1 = kp1[idx1][0], kp1[idx1][1]
		x2, y2 = kp2[idx2][0] + w1, kp2[idx2][1]
		
		# Use a different color for each line
		plt.plot([x1, x2], [y1, y2], color=colors[i])
		
		# Draw small circles at each vertex
		plt.plot(x1, y1, 'o', color=colors[i], markersize=5)
		plt.plot(x2, y2, 'o', color=colors[i], markersize=5)
	
	plt.axis('off')
	plt.title("Matches: " + str(len(matches))) # Use f-string formatting
	plt.tight_layout()
	plt.show()

def sift(gray_image, num_octaves=4, scales_per_octave=4, contrast_threshold=0.6, edge_threshold=10):
    """
    SIFT feature detection and description pipeline.
    
    Args:
        image: Input image
        num_octaves: Number of octaves
        scales_per_octave: Number of scales per octave
        contrast_threshold: Threshold for low contrast keypoints
        edge_threshold: Threshold for edge response
        
    Returns:
        keypoints: List of keypoint locations
        descriptors: Array of descriptors
    """
    # If not a grayscale image, convert it
    if len(gray_image.shape) > 2:
        gray_image = cv2.cvtColor(gray_image, cv2.COLOR_RGB2GRAY)
        
		# If not a float image, convert it
    if gray_image.dtype != np.float32:
        gray_image = gray_image.astype(np.float32)
    
    # Create Gaussian pyramid
    gaussian_pyr = create_gaussian_pyramid(gray_image, num_octaves, scales_per_octave)
    
    # Create DoG pyramid
    dog_pyr = create_dog_pyramid(gaussian_pyr)
    
    # Detect keypoints
    keypoints = detect_keypoints2(dog_pyr, contrast_threshold, edge_threshold)
    
    # Compute orientations
    oriented_keypoints = compute_orientations2(gaussian_pyr, keypoints)
    
    # Compute descriptors
    keypoints_list, descriptors = compute_descriptors2(gaussian_pyr, oriented_keypoints)
    
    return keypoints_list, descriptors



## 1. Load Images

Let's load two images to test our SIFT implementation. You can replace these with your own images.

In [ ]:
# Load your images
# Replace these paths with the path to your own images
img1_path = 'roof1.png'  # Replace with your image path
img2_path = 'roof2.png'  # Replace with your image path
img3_path = 'school_gate.jpeg'  # Replace with your image path

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)
img3 = cv2.imread(img3_path)

# Convert from BGR to RGB for visualization
img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img3_rgb = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)

# Display the images
plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.imshow(img1_rgb)
plt.title('Image 1')
plt.axis('off')

plt.subplot(122)
plt.imshow(img2_rgb)
plt.title('Image 2')
plt.axis('off')

plt.tight_layout()
plt.show()

## Change the images to grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3_gray = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

# show the grayscale images
plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.imshow(img1_gray, cmap='gray')
plt.title('Image 1')

plt.subplot(122)
plt.imshow(img2_gray, cmap='gray')
plt.title('Image 2')

plt.tight_layout()
plt.show()



## 2. Detect SIFT Features

Now we'll detect SIFT features in both images using our custom implementation.

In [ ]:
# Parameters for SIFT
num_octaves = 4
scales_per_octave = 4
contrast_threshold = 0.7
edge_threshold = 10

img1_gray = img1_gray.astype(np.float32)
img2_gray = img2_gray.astype(np.float32)
img3_gray = img3_gray.astype(np.float32)
# Detect SIFT features in the first image
print("Detecting SIFT features in image 1...")
keypoints1, descriptors1 = sift(img1_gray, num_octaves, scales_per_octave, contrast_threshold, edge_threshold)
print(f"Found {len(keypoints1)} keypoints in image 1")

# Detect SIFT features in the second image
print("\nDetecting SIFT features in image 2...")
keypoints2, descriptors2 = sift(img2_gray, num_octaves, scales_per_octave, contrast_threshold, edge_threshold)
print(f"Found {len(keypoints2)} keypoints in image 2")

keypoints3, descriptors3 = sift(img3_gray, num_octaves, scales_per_octave, contrast_threshold, edge_threshold)
print(f"Found {len(keypoints3)} keypoints in image 3")

## 3. Visualize Detected Keypoints

Let's visualize the detected keypoints on both images.

In [ ]:
# Visualize keypoints on the first image
print("Keypoints in image 1: numbers of keypoints: ", len(keypoints1))
visualize_keypoints(img1_rgb, keypoints1)

# Visualize keypoints on the second image
print("Keypoints in image 2: numbers of keypoints: ", len(keypoints2))
visualize_keypoints(img2_rgb, keypoints2)

print("Keypoints in image 3: numbers of keypoints: ", len(keypoints3))
visualize_keypoints(img3_rgb, keypoints3)

## 4. Match Keypoints Between Images

Now let's match keypoints between the two images using the ratio test.

In [ ]:
# Match descriptors between images
ratio_threshold = 0.85  # Lowe's ratio test threshold
matches = match_descriptors(descriptors1, descriptors2, ratio_threshold)

print(f"Found {len(matches)} matches between the images")

## 5. Visualize Matches

Finally, let's visualize the matches between the two images.

In [ ]:
# Visualize the matches
visualize_matches(img1_gray, keypoints1, img2_gray, keypoints2, matches)

## 6. Experiment: Effect of Different Parameters

Let's see how changing the parameters affects the SIFT detection and matching results.

In [ ]:
# Different parameters to try
contrast_thresholds = [0.55, 0.75, 0.8]
edge_thresholds = [5, 10, 15]

# A sample experiment with different contrast thresholds
for threshold in contrast_thresholds:
    print(f"\nUsing contrast threshold = {threshold}")
    
    # Detect features with this threshold
    kp1, desc1 = sift(img1_gray, num_octaves, scales_per_octave, threshold, edge_threshold)
    kp2, desc2 = sift(img2_gray, num_octaves, scales_per_octave, threshold, edge_threshold)
    
    print(f"Image 1: {len(kp1)} keypoints")
    print(f"Image 2: {len(kp2)} keypoints")
    
    # Match the features
    matches = match_descriptors(desc1, desc2, ratio_threshold)
    print(f"Matches: {len(matches)}")
    
    # Visualize a few keypoints (just to avoid too many plots)
    if len(kp1) > 0 and len(kp2) > 0 and len(matches) > 0:
        visualize_matches(img1_rgb, kp1, img2_rgb, kp2, matches[:50] if len(matches) > 50 else matches)

## 7. Try with Different Images

You can replace the image paths at the beginning of this notebook to try with different images. Here's a function to make it easier to test with new images.

In [ ]:
def test_sift_on_images(img1_path, img2_path, num_octaves=4, scales_per_octave=4, 
                        contrast_threshold=0.6, edge_threshold=10, ratio_threshold=0.75):
    """Test SIFT feature detection and matching on two images"""
    # Load images
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    
    # Convert to RGB
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    
    # Display images
    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plt.imshow(img1_rgb)
    plt.title('Image 1')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(img2_rgb)
    plt.title('Image 2')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Detect SIFT features
    print("Detecting features...")
    img1_rgb = img1_rgb.astype(np.float32)
    img2_rgb = img2_rgb.astype(np.float32)
    kp1, desc1 = sift(img1_rgb, num_octaves, scales_per_octave, contrast_threshold, edge_threshold)
    kp2, desc2 = sift(img2_rgb, num_octaves, scales_per_octave, contrast_threshold, edge_threshold)
    
    print(f"Found {len(kp1)} keypoints in image 1")
    print(f"Found {len(kp2)} keypoints in image 2")
    
    # Visualize keypoints
    img1_rgb = img1_rgb.astype(np.uint8)
    img2_rgb = img2_rgb.astype(np.uint8)
    visualize_keypoints(img1_rgb, kp1)
    visualize_keypoints(img2_rgb, kp2)
    
    # Match features
    matches = match_descriptors(desc1, desc2, ratio_threshold)
    print(f"Found {len(matches)} matches between the images")
    
    # Visualize matches
    visualize_matches(img1_rgb, kp1, img2_rgb, kp2, matches)
    
    return kp1, desc1, kp2, desc2, matches

# You can use this function to test with your own images
#kp1, desc1, kp2, desc2, matches = test_sift_on_images('books1.jpeg', 'books2.jpeg', num_octaves=4, scales_per_octave=4,
#																											contrast_threshold=0.6, edge_threshold=10, ratio_threshold=0.8)

In [ ]:
kp1, desc1, kp2, desc2, matches = test_sift_on_images('building_2.jpg', 'building_2d.jpg', num_octaves=4, scales_per_octave=4,
																											contrast_threshold=0.6, edge_threshold=10, ratio_threshold=0.75)

In [ ]:
# use ransac
img1_rgb = cv2.cvtColor(cv2.imread('building_2.jpg'), cv2.COLOR_BGR2RGB)
img2_rgb = cv2.cvtColor(cv2.imread('building_2d.jpg'), cv2.COLOR_BGR2RGB)
inliers, homography = ransac_match(kp1, kp2, matches, num_iterations=1000, inlier_threshold=7)
print("original matches: ", len(matches))
print(f"Found {len(inliers)} inliers")
print("Homography matrix:")
print(homography)
visualize_matches(img1_rgb, kp1, img2_rgb, kp2, matches)
visualize_matches(img1_rgb, kp1, img2_rgb, kp2, inliers)

In [ ]:
kp1, desc1, kp2, desc2, matches = test_sift_on_images('mars1.jpg', 'mars2.jpg', num_octaves=4, scales_per_octave=4,
																											contrast_threshold=0.6, edge_threshold=10, ratio_threshold=0.75)

## 8. Rotation and Scale Invariance Test

Let's test whether our SIFT implementation is truly invariant to rotation and scaling by applying transformations to one of our images.

In [ ]:
def test_invariance(img_path):
    """Test rotation and scale invariance of SIFT"""
    # Load the original image
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Create a rotated version (45 degrees)
    rows, cols = img.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((cols/2, rows/2), 45, 1)
    img_rotated = cv2.warpAffine(img, rotation_matrix, (cols, rows))
    img_rotated_rgb = cv2.cvtColor(img_rotated, cv2.COLOR_BGR2RGB)
    #save the rotation image
    cv2.imwrite("building_2r45.jpg", img_rotated)
    
    # Create a scaled version (0.75x)
    img_scaled = cv2.resize(img, None, fx=0.75, fy=0.75)
    img_scaled_rgb = cv2.cvtColor(img_scaled, cv2.COLOR_BGR2RGB)
    
    # Display the three images
    plt.figure(figsize=(15, 5))
    
    plt.subplot(131)
    plt.imshow(img_rgb)
    plt.title('Original')
    plt.axis('off')
    
    plt.subplot(132)
    plt.imshow(img_rotated_rgb)
    plt.title('Rotated 45°')
    plt.axis('off')
    
    plt.subplot(133)
    plt.imshow(img_scaled_rgb)
    plt.title('Scaled 0.75x')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Test with original vs. rotated
    print("\nTesting Original vs. Rotated:")
    # change to float32
    img_rgb = img_rgb.astype(np.float32)
    img_rotated_rgb = img_rotated_rgb.astype(np.float32)
    img_scaled_rgb = img_scaled_rgb.astype(np.float32)
    
    kp_orig, desc_orig = sift(img_rgb, num_octaves=4, scales_per_octave=4, contrast_threshold=0.6, edge_threshold=10)
    kp_rot, desc_rot = sift(img_rotated_rgb, num_octaves=4, scales_per_octave=4, contrast_threshold=0.6, edge_threshold=10)
    img_rgb = img_rgb.astype(np.uint8)
    img_rotated_rgb = img_rotated_rgb.astype(np.uint8)
    visualize_keypoints(img_rotated_rgb, kp_rot)
    visualize_keypoints(img_rgb, kp_orig)
    matches_rot = match_descriptors(desc_orig, desc_rot, ratio_threshold=0.75)
    print(f"Found {len(matches_rot)} matches between original and rotated images")
    visualize_matches(img_rgb, kp_orig, img_rotated_rgb, kp_rot, matches_rot)
    
    # Test with original vs. scaled
    print("\nTesting Original vs. Scaled:")
    kp_scaled, desc_scaled = sift(img_scaled_rgb, num_octaves=4, scales_per_octave=4, contrast_threshold=0.6, edge_threshold=10)
    matches_scaled = match_descriptors(desc_orig, desc_scaled, ratio_threshold=0.75)
    img_scaled_rgb = img_scaled_rgb.astype(np.uint8)
    visualize_keypoints(img_scaled_rgb, kp_scaled)
    print(f"Found {len(matches_scaled)} matches between original and scaled images")
    visualize_matches(img_rgb, kp_orig, img_scaled_rgb, kp_scaled, matches_scaled)

# You can use this function to test invariance with your own image
test_invariance('building_2.jpg')

## 9. Performance Comparison with OpenCV SIFT

Let's compare our SIFT implementation with OpenCV's built-in SIFT in terms of keypoint detection and matching.

In [ ]:
def compare_with_opencv_sift(img1_path, img2_path):
    """Compare our SIFT implementation with OpenCV's built-in SIFT"""
    # Load images
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    
    # Our SIFT implementation
    print("\nUsing our SIFT implementation:")
    
    # Measure time
    import time
    start_time = time.time()
    
    kp1_our, desc1_our = sift(img1_rgb)
    kp2_our, desc2_our = sift(img2_rgb)
    matches_our = match_descriptors(desc1_our, desc2_our)
    
    our_time = time.time() - start_time
    
    print(f"Found {len(kp1_our)} keypoints in image 1")
    print(f"Found {len(kp2_our)} keypoints in image 2")
    print(f"Found {len(matches_our)} matches between images")
    print(f"Time taken: {our_time:.2f} seconds")
    
    # OpenCV SIFT implementation
    print("\nUsing OpenCV SIFT implementation:")
    
    # Create OpenCV SIFT detector
    sift_cv = cv2.SIFT_create()
    
    # Measure time
    start_time = time.time()
    
    # Detect and compute with OpenCV SIFT
    kp1_cv, desc1_cv = sift_cv.detectAndCompute(img1, None)
    kp2_cv, desc2_cv = sift_cv.detectAndCompute(img2, None)
    
    # Match with OpenCV BFMatcher
    bf = cv2.BFMatcher()
    matches_cv = bf.knnMatch(desc1_cv, desc2_cv, k=2)
    
    # Apply ratio test
    good_matches = []
    for m, n in matches_cv:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    
    cv_time = time.time() - start_time
    
    print(f"Found {len(kp1_cv)} keypoints in image 1")
    print(f"Found {len(kp2_cv)} keypoints in image 2")
    print(f"Found {len(good_matches)} matches between images")
    print(f"Time taken: {cv_time:.2f} seconds")
    
    # Display OpenCV SIFT matches
    plt.figure(figsize=(12, 8))
    plt.title('OpenCV SIFT Matches')
    img_matches = cv2.drawMatches(img1, kp1_cv, img2, kp2_cv, good_matches, None, flags=2)
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Display our SIFT matches
    print("\nOur SIFT Implementation Matches:")
    visualize_matches(img1_rgb, kp1_our, img2_rgb, kp2_our, matches_our)

# You can use this function to compare implementations
compare_with_opencv_sift('books1.jpeg', 'books2.jpeg')